# PATH

구글 드라이브

In [8]:
import os
import sys
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
os.chdir('/content/drive/MyDrive/0901_RNN/보충/RNN 과제')

In [10]:
os.getcwd()

'/content/drive/MyDrive/0901_RNN/보충/RNN 과제'

In [11]:
root_data = f"{os.getcwd()}/data/data_hw"
root_data

'/content/drive/MyDrive/0901_RNN/보충/RNN 과제/data/data_hw'

In [12]:
# 데이터 불러올 때 사용할 경로로 이후에 있을 unzip과 연동되는 내용입니다! 
path_train_data = "data/data_hw/1.Training/원천데이터/일상생활및구어체_영한_train_set.json"

path_val_data = "data/data_hw/2.Validation/원천데이터/일상생활및구어체_영한_valid_set.json"


# 사전 준비 사항

In [13]:
import torch

device = 'cuda' if torch.cuda.is_available() else "cpu"
device

'cuda'

# 1. Data

실습에서 사용했던 영어-프랑스어와 달리 현재 사용할 데이터 셋은 영어-한국어 데이터 셋입니다. 

특히 압축을 풀어야 사용하실 수 있으니 유의하여 진행해주시길 바랍니다.

다행인 점은 용량이 그리 크지 않을겁니다!

저장 경로는 위에서 path_train_data와 path_val_data와 동일하게 맞췄습니다.

In [14]:
!unzip -qq data/data_hw/1.Training/원천데이터/TS1.zip -d data/data_hw/1.Training/원천데이터

!unzip -qq data/data_hw/2.Validation/원천데이터/VS1.zip -d data/data_hw/2.Validation/원천데이터


## 데이터 구조 확인

In [15]:
import json

In [16]:
# 좀 오래 걸릴 수 있다.

# 경로는 앞서 PATH에서 선언했고 unzip에서도 경로를 지정했습니다. 만약 여기에 변화가 생기신다면 에러가 생길 수 있으니 유의하셔야 합니다.
with open(path_train_data, 'r') as f:
  raw_train_data = json.load(f) # 학습데이터

with open(path_val_data, 'r') as f:
  raw_val_data = json.load(f) # 평가데이터/테스트데이터


In [17]:
# 현재 데이터의 구조가 {'data':~~~ }로 되어있는것을 알 수 있습니다.
raw_train_data.keys()

dict_keys(['data'])

In [18]:
# 학습 데이터 개수 확인
len(raw_train_data['data'])

1200307

지금 데이터가 너무 많아서 전처리만 해도 1,2시간 걸릴 수 있습니다. 그래서 숙제에서는 매우 일부 데이터만(10,000개) 이용하도록 하겠습니다.

In [19]:
raw_train_data = raw_train_data['data'][:10000]
raw_val_data = raw_val_data['data'][:1000]

In [20]:
# 샘플로 하나 확인
sample_data = raw_train_data[0]

In [21]:
# 아래의 예시를 보고 데이터를 어떻게 처리할지 잘 생각하셔야 합니다!
# 보았을 때 우리가 사용해야 하는 건 ko와 en가 되고
# 실제 데이터를 사용한다고 할 때 이 외에 필요한 데이터가 더 있을 수 있습니다.
# 모델링 과정에서는 어떻게 데이터들이 저장되어 있는지 주의해서 처리하시면 됩니다.
sample_data

{'sn': 'ECOAR1A00003',
 'data_set': '일상생활및구어체',
 'domain': '해외고객과의채팅',
 'subdomain': '숙박,음식점',
 'en_original': "I'm glad to hear that, and I hope you do consider doing business with us.",
 'en': "I'm glad to hear that, and I hope you do consider doing business with us.",
 'mt': '그 소식을 들으니 기쁩니다. 우리와 거래하는 것을 고려해 보시기 바랍니다.',
 'ko': '그 말을 들으니 기쁘고, 저희와 거래하는 것을 고려해 주셨으면 합니다.',
 'source_language': 'en',
 'target_language': 'ko',
 'word_count_ko': 10.0,
 'word_count_en': 15.0,
 'word_ratio': 0.667,
 'file_name': '해외고객과의채팅_숙박,음식점.xlsx',
 'source': '크라우드 소싱',
 'license': 'open',
 'style': '구어체',
 'included_unknown_words': False,
 'ner': None}

# 2. Preprocess

세션 설명 코드에서는 알파벳 하나하나 사용했지만, 이번 과정에서는 토큰화를 진행하여 사용하는 과정을 Dataset에 바로 적용해보고자 합니다! 

토큰화는 앞서 word embedding 세션에서 사용한 것들을 그대로 이용하고자 합니다. 세부 커스터마이징은 자유롭습니다.

전처리 과정에서 중요한 점은, 앞의 구현 코드에서 여러 개의 for loop을 이용해서 구현했었지만, 실제 데이터를 사용한다면 매우 오래 걸리게 됩니다. 실제 데이터를 사용할 때엔 결국 얼마나 효율적으로 빠르게 정리할 수 있을지 고민해보시면서 접근하시길 바랍니다.

즉, 전처리에서 챙길 통찰은 얼마나 "효율적"으로 할 수 있는가? 입니다.

## Tokenizer

지금은 단순히 품사를 기준으로 문장 그대로 단어로 들어가서 학습이 진행된다.

그러나 이후에는 같은 품사끼리(예를 들면, 한국어의 동사와 영어의 동사끼리) 매칭하면 훨씬 정확히 학습을 할 수 있지 않을까 싶다.

추가로 stem,norm 등을 적용하여 품사의 형태를 바꿔주면 더욱 학습 효과가 높을 것이다.(예를 들면, gone -> go)

### 1. 한국어 토큰화

In [ ]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [23]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [24]:
from konlpy.tag import Okt

In [25]:
okt = Okt()

In [26]:
okt.pos(sample_data['ko'])

[('그', 'Noun'),
 ('말', 'Noun'),
 ('을', 'Josa'),
 ('들으니', 'Verb'),
 ('기쁘고', 'Adjective'),
 (',', 'Punctuation'),
 ('저희', 'Noun'),
 ('와', 'Josa'),
 ('거래', 'Noun'),
 ('하는', 'Verb'),
 ('것', 'Noun'),
 ('을', 'Josa'),
 ('고려', 'Noun'),
 ('해', 'Verb'),
 ('주셨으면', 'Verb'),
 ('합니다', 'Verb'),
 ('.', 'Punctuation')]

In [27]:
stop_words = []

def CustomTokenizer_ko(sentence):

    # stop_words = ['하다','다', '것', '의하다', '돼다', '수', '등'] # 의미없는 단어만을 담은 리스트
    stop_pos = ['KoreanParticle'] # 의미없는 품사를 담은 리스트
    result = [] # 추출한 단어 중 의미있는 것들을 담은 리스트

    # Punctuation : 맞춤표, 쉼표
    # Alpha : 
    # Suffix : 접미사 ex. 먹보에서 -보
    # KoreanParticle : ex. ㅋㅋㅋ

    for word,char in okt.pos(sentence, stem = False, norm = False):
      # stem, norm모두 False로 하여 최대한 재가공을 하지 않는다.
      # 왜냐하면 영어를 토큰화할 때, 사용하는 nltk는 위 기능을 제공하지 않는 것 같기 때문이다.
      # 따라서 nltk와 같게 그냥 문장 그대로를 끊어주기만 하자!

      if char not in stop_pos: # 의미없는 품사 제외
        if word not in stop_words: # 의미없는 단어 제외
          result.append(word)
    
    #######
    # 띄어쓰기를 넣어주어야

    return result

In [28]:
# 한국어 처리 어떻게 되고 있는지 확인
CustomTokenizer_ko(sample_data['ko'])

['그',
 '말',
 '을',
 '들으니',
 '기쁘고',
 ',',
 '저희',
 '와',
 '거래',
 '하는',
 '것',
 '을',
 '고려',
 '해',
 '주셨으면',
 '합니다',
 '.']

### 2. 영어 토큰화

In [29]:
# 설치시에 시간이 좀 걸린다.

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
nltk.download("book", quiet=True)
from nltk.book import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [30]:
sentence = sample_data['en']
tagged_list = pos_tag(word_tokenize(sentence))
tagged_list

[('I', 'PRP'),
 ("'m", 'VBP'),
 ('glad', 'JJ'),
 ('to', 'TO'),
 ('hear', 'VB'),
 ('that', 'IN'),
 (',', ','),
 ('and', 'CC'),
 ('I', 'PRP'),
 ('hope', 'VBP'),
 ('you', 'PRP'),
 ('do', 'VBP'),
 ('consider', 'VB'),
 ('doing', 'VBG'),
 ('business', 'NN'),
 ('with', 'IN'),
 ('us', 'PRP'),
 ('.', '.')]

In [31]:
# 필요없는 품사 제거

# ------------------------------------
# NNP: 단수 고유명사
# VB: 동사
# VBP: 동사 현재형
# TO: to 전치사
# NN: 명사(단수형 혹은 집합형)
# DT: 관형사
# ------------------------------------

stop_pos = [] # ',','NN' 등 제거할 품사를 넣는다.

nouns_list = [t[0] for t in tagged_list if t[1] not in []]
nouns_list

['I',
 "'m",
 'glad',
 'to',
 'hear',
 'that',
 ',',
 'and',
 'I',
 'hope',
 'you',
 'do',
 'consider',
 'doing',
 'business',
 'with',
 'us',
 '.']

In [32]:
word_tokenize(sentence+'\n')

['I',
 "'m",
 'glad',
 'to',
 'hear',
 'that',
 ',',
 'and',
 'I',
 'hope',
 'you',
 'do',
 'consider',
 'doing',
 'business',
 'with',
 'us',
 '.']

In [33]:
def CustomTokenizer_en(sentence):
  tagged_list = pos_tag(word_tokenize(sentence))
  nouns_list = [t[0] for t in tagged_list if t[1] not in []]
  
  return nouns_list

In [34]:
CustomTokenizer_en(sample_data['en'])

['I',
 "'m",
 'glad',
 'to',
 'hear',
 'that',
 ',',
 'and',
 'I',
 'hope',
 'you',
 'do',
 'consider',
 'doing',
 'business',
 'with',
 'us',
 '.']

In [35]:
# 참고로 nltk이외에도 keras에서 제공하는 기능도 있다.

from tensorflow.keras.preprocessing.text import text_to_word_sequence as CustomTokenizer_en1

CustomTokenizer_en1(sample_data['en'])


["i'm",
 'glad',
 'to',
 'hear',
 'that',
 'and',
 'i',
 'hope',
 'you',
 'do',
 'consider',
 'doing',
 'business',
 'with',
 'us']

## 본격적인 전처리 과정

전처리릏 할 때 주의할 점은, 우리가 가지고 있는 것은 train 데이터 뿐이라는 점입니다.

토큰화를 하고, token들을 수집할 때 평가데이터(혹은 테스트 데이터)에 대한 정보는 가지고 있을 수 없습니다. 그렇기 때문에 train data를 기준으로 전처리가 진행되어야 합니다.

### Tokenization

In [36]:
from tqdm import tqdm

In [37]:
# 시간이 좀 걸린다.

src_vocab = set() # source 즉, 유일한 '한국어' 단어
tar_vocab = set() # target 즉, 유일한 '영어' 단어

src_seq_tr = [] # 전체 문장; 한국어
tar_seq_tr = [] # 전체 문장; 영어

# 전체 raw data에서 문장하나씩 loop 돌아가는 것
for raw_tr_dat in tqdm(raw_train_data):

  # 문장별 tokenization
  src_tmp = CustomTokenizer_ko(raw_tr_dat['ko'])
  tar_tmp = CustomTokenizer_en(raw_tr_dat['en'])

  # 입력 시퀀스에서 문장의 끝을 알리는 <EOS> 토큰 추가; 한국어
  src_tmp = src_tmp + ['\n']

  # 출력 시퀀스에서 시작과 끝을 알리는 <SOS>, <EOS> 토큰 추가; 영어
  tar_tmp = ['\t'] + tar_tmp + ['\n']

  # 한국어, 영어 단어집합 구성; 리스트를 세트에 추가하는 것임으로 update()를 사용한다.
  src_vocab.update(src_tmp)
  tar_vocab.update(tar_tmp)

  # tokenization 끝난 문장 보관
  src_seq_tr.append(src_tmp)
  tar_seq_tr.append(tar_tmp)

  

100%|██████████| 10000/10000 [00:47<00:00, 210.92it/s]


In [38]:
print('unique한 한국어 단어 수 :', len(src_vocab))
print('unique한 영어 단어 수 :', len(tar_vocab))

unique한 한국어 단어 수 : 8865
unique한 영어 단어 수 : 7293


In [39]:
src_seq_tr[:3] # 입력인 한국어의 경우에 뒤에 '\n'이 잘 붙어 있다.

[['그',
  '말',
  '을',
  '들으니',
  '기쁘고',
  ',',
  '저희',
  '와',
  '거래',
  '하는',
  '것',
  '을',
  '고려',
  '해',
  '주셨으면',
  '합니다',
  '.',
  '\n'],
 ['확실히',
  '생각',
  '하고',
  '있습니다만',
  ',',
  '몇',
  '가지',
  '여쭤',
  '보고',
  '싶은',
  '게',
  '있어요',
  '.',
  '\n'],
 ['오늘날',
  '세계',
  '5',
  '가구',
  '중',
  '1',
  '가구',
  '는',
  '고양이',
  '나',
  '개',
  '또는',
  '둘',
  '다를',
  '키우고',
  '있습니다',
  '.',
  '\n']]

In [40]:
tar_seq_tr[:3] # 입력인 영어의 경우에 앞에는 '\t'가 뒤에는 '\n'이 잘 붙어 있다.

[['\t',
  'I',
  "'m",
  'glad',
  'to',
  'hear',
  'that',
  ',',
  'and',
  'I',
  'hope',
  'you',
  'do',
  'consider',
  'doing',
  'business',
  'with',
  'us',
  '.',
  '\n'],
 ['\t',
  'I',
  "'m",
  'definitely',
  'thinking',
  'about',
  'it',
  ',',
  'but',
  'I',
  'have',
  'some',
  'queries',
  'to',
  'ask',
  'you',
  '.',
  '\n'],
 ['\t',
  'In',
  'today',
  "'s",
  'world',
  ',',
  'one',
  'in',
  'every',
  'five',
  'families',
  'has',
  'either',
  'a',
  'cat',
  ',',
  'dog',
  ',',
  'or',
  'both',
  '.',
  '\n']]

* 고민해볼 점: 과연 한국어 단어집합에 한국어만 있을까요?

* 답변 : 한국어에 영어가 있어도, 아주 복잡한 형태가 아닌 이상 괜찮을 것도 같다.

In [41]:
s = '나는 너를 love한다.'
CustomTokenizer_ko(s)

['나', '는', '너', '를', 'love', '한다', '.']

### token - index 정리

In [43]:
# 최종 정수 인코딩 하기 전 보기 편하기 위해 sort
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

In [44]:
# token - idx dictionary
# 여기서도 동일하게 i+1로 해줘야 padding에 사용할 token을 0으로 사용할 수 있습니다!

# src_vocab, tar_vocab과 같이 unique한 단어들만 있는 리스트를
# 앞에 숫자를 포함한 딕셔너리로 만든 변수
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

print(src_to_index) # '\n'이 1번쨰이다.
print(tar_to_index) # '\t', '\n' 1,2번째이다.

{'\n': 1, '!': 2, '"': 3, '$10,000': 4, '$10,000만': 5, '$100': 6, '$100,000': 7, '$120': 8, '$20': 9, '$21': 10, '$5': 11, '$50,000': 12, '$500,000': 13, '$6,000': 14, "'": 15, '(': 16, ')': 17, ',': 18, '-': 19, '.': 20, '0': 21, '0.006': 22, '0.2': 23, '0.26%': 24, '0.5%': 25, '0.95%': 26, '0000000': 27, '1': 28, '1%': 29, '1,000': 30, '1,000,000원': 31, '1,000달러': 32, '1,000만': 33, '1,000원': 34, '1,200달러': 35, '1,250,000원': 36, '1,500': 37, '1,500달러': 38, '1,700,000원': 39, '1.50%': 40, '10': 41, '10%': 42, '10,000': 43, '10,000달러': 44, '10.9%': 45, '100': 46, '100%': 47, '100,000달러': 48, '1000': 49, '1000만': 50, '1000억원': 51, '100달러': 52, '100만': 53, '100만달러': 54, '100만원': 55, '10~14일': 56, '10년': 57, '10달러': 58, '10만': 59, '10분': 60, '10시': 61, '10월': 62, '10일': 63, '10초': 64, '11': 65, '1100': 66, '11시': 67, '11월': 68, '11일': 69, '12': 70, '12%': 71, '12.99달러': 72, '1200만': 73, '123': 74, '123-456': 75, '1234': 76, '1234-5678': 77, '123456': 78, '1234567': 79, '12345678': 80, '1234

In [45]:
src_to_index['\n'] # source에서는 '<EOS>'가 '\n'으로써 1로 레이블된다.

1

In [47]:
tar_to_index['\n'] # target에서는 '<EOS>'가 '\n'으로써 2로 레이블된다.

2

In [48]:
tar_to_index['\t'] # target에서는 '<SOS>'가 2'\t'로써 1로 레이블된다.

1

### 정수 인코딩

In [ ]:
# 앞서 toekn-index dictionary로 정리한 것을 이용해서 각각의 토큰들을 정수인코딩을 진행해봅니다.
# 이 과정에서 유의할 점은, 기존의 구현 코드에서 여러개의 for loop을 사용했습니다.
# 이보다 빨리 진행하기 위해 어떻게 코딩할 수 있을까요?
# 시간이 충분하시다면 for loop을 그대로 이용하셔도 좋습니다만, 다른 방법도 고민하셔도 좋습니다.
# hint: Pandas에서 apply와 비슷한 것들을 이용해보면 어떨까요?

In [49]:
# 인코딩 방법

list(map(lambda x : src_to_index[x], src_seq_tr[0]))

[1219,
 2837,
 5996,
 2454,
 1390,
 18,
 6634,
 5691,
 774,
 8306,
 861,
 5996,
 979,
 8497,
 7053,
 8476,
 20,
 1]

In [50]:
# 이제 문장을 잘라 만든 단어들의 이중리스트인 'src_seq_tr'와 'tar_seq_tr'를
# 문자에서 숫자로 인코딩을 해주자!

# 정수 인코딩 값이 들어갈 장소
encoder_input = []
decoder_temp = []
decoder_input = []
decoder_target = []

# encoder의 입력값 정수 인코딩 진행
for sentence in src_seq_tr:
  encoder_input.append(list(map(lambda x : src_to_index[x], sentence)))

# 우선 decoder에서 사용하는 문장에 대해 정수인코딩 진행
for sentence in tar_seq_tr:
  decoder_temp.append(list(map(lambda x : tar_to_index[x], sentence)))

# 다만 decoder의 출력값은 <SOS>들어가면 안되니 첫번째 토큰은 빼주기
decoder_target = list(map(lambda x : x[1:], decoder_temp))
# decoder의 입력값은 마지막 <EOS>가 들어가면 안되니 마지막 토큰은 빼주기
decoder_input = list(map(lambda x : x[:-1], decoder_temp))

# 마지막으로 이제는 tensor의 형태로 바꿔준다.
encoder_input = list(map(lambda x : torch.Tensor(x).long(), encoder_input))
decoder_input = list(map(lambda x : torch.Tensor(x).long(), decoder_input))
decoder_target = list(map(lambda x : torch.Tensor(x).long(), decoder_target))

In [51]:
encoder_input[:3] # 모두 1('\n')로 끝난다.

[tensor([1219, 2837, 5996, 2454, 1390,   18, 6634, 5691,  774, 8306,  861, 5996,
          979, 8497, 7053, 8476,   20,    1]),
 tensor([8764, 4209, 8285, 6391,   18, 3030,  583, 5460, 3574, 4836,  864, 6397,
           20,    1]),
 tensor([5589, 4339,  246,  475, 7115,   28,  475, 1829,  992, 1517,  730, 2535,
         2312, 1881, 7809, 6390,   20,    1])]

In [52]:
decoder_input[:3] # 모두 1('\t')로 시작한다.

[tensor([   1,  573,   10, 3540, 6712, 3689, 6636,   16, 1277,  573, 3766, 7281,
         2744, 2253, 2757, 1789, 7213, 6964,   17]),
 tensor([   1,  573,   10, 2555, 6661, 1040, 4088,   16, 1795,  573, 3678, 6192,
         5436, 6712, 1387, 7281,   17]),
 tensor([   1,  584, 6717,   12, 7240,   16, 4812, 3880, 3063, 3321, 3209, 3674,
         2898, 1034, 1893,   16, 2755,   16, 4853, 1701,   17])]

In [53]:
decoder_target[:3] # 모두 2('\n')로 끝난다.

[tensor([ 573,   10, 3540, 6712, 3689, 6636,   16, 1277,  573, 3766, 7281, 2744,
         2253, 2757, 1789, 7213, 6964,   17,    2]),
 tensor([ 573,   10, 2555, 6661, 1040, 4088,   16, 1795,  573, 3678, 6192, 5436,
         6712, 1387, 7281,   17,    2]),
 tensor([ 584, 6717,   12, 7240,   16, 4812, 3880, 3063, 3321, 3209, 3674, 2898,
         1034, 1893,   16, 2755,   16, 4853, 1701,   17,    2])]

### Padding

In [54]:
import numpy as np

mean_encode_len = np.array([len(line) for line in encoder_input]).mean()
mean_decode_input_len = np.array([len(line) for line in decoder_input]).mean()
mean_decode_target_len = np.array([len(line) for line in decoder_target]).mean()

print('encode 한 문장안에서의 평균 단어의 갯수 :',mean_encode_len)
print('decode-input 한 문장안에서의 평균 단어의 갯수 :',mean_decode_input_len)
print('decode-output 한 문장안에서의 평균 단어의 갯수 :',mean_decode_target_len)

# 문장을 나눈 단어의 평균 갯수의 크게 차이가 없다.
# 사이즈의 차이가 없는 것은 아마도 비슷한 기준으로 문장을 나눴기 때문일 것이다.

# 그러나 중요한 것은 encode/decode 내에서 각 문장의 길이가 같도록 맞춰줘야하는 것이다.
# 따라서 아래 과정에 따라 0을 집어넣어 사이즈를 맞춰주는 padding을 진행해보자!

encode 한 문장안에서의 평균 단어의 갯수 : 12.426
decode-input 한 문장안에서의 평균 단어의 갯수 : 13.492
decode-output 한 문장안에서의 평균 단어의 갯수 : 13.492


In [55]:
from torch.nn.utils.rnn import pad_sequence

In [56]:
# 현재 길이가 제각각인 정수화된 문장이 있습니다. padding을 통해 길이를 맞춰봅시다
# 주의사항: 여기에 사용된 값을은 정수일까요?
batch_first = True

# _tr이 붙으면, transformation 즉, padding이 된 상태이다.
encoder_input_tr = pad_sequence(encoder_input, batch_first=batch_first).long()
decoder_input_tr = pad_sequence(decoder_input, batch_first=batch_first).long()
decoder_target_tr = pad_sequence(decoder_target, batch_first=batch_first).long()

In [57]:
encoder_input_tr[:3]

tensor([[1219, 2837, 5996, 2454, 1390,   18, 6634, 5691,  774, 8306,  861, 5996,
          979, 8497, 7053, 8476,   20,    1,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [8764, 4209, 8285, 6391,   18, 3030,  583, 5460, 3574, 4836,  864, 6397,
           20,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [5589, 4339,  246,  475, 7115,   28,  475, 1829,  992, 1517,  730, 2535,
         2312, 1881, 7809, 6390,   20,    1,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0]])

In [58]:
decoder_input_tr[:3]

tensor([[   1,  573,   10, 3540, 6712, 3689, 6636,   16, 1277,  573, 3766, 7281,
         2744, 2253, 2757, 1789, 7213, 6964,   17,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [   1,  573,   10, 2555, 6661, 1040, 4088,   16, 1795,  573, 3678, 6192,
         5436, 6712, 1387, 7281,   17,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [   1,  584, 6717,   12, 7240,   16, 4812, 3880, 3063, 3321, 3209, 3674,
         2898, 1034, 1893,   16, 2755,   16, 4853, 1701,   17,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]])

In [59]:
mean_encode_len = np.array([len(line) for line in encoder_input_tr]).mean()
mean_decode_input_len = np.array([len(line) for line in decoder_input_tr]).mean()
mean_decode_target_len = np.array([len(line) for line in decoder_target_tr]).mean()

print('encode 한 문장안에서의 평균 단어의 갯수 :',mean_encode_len)
print('target-input 한 문장안에서의 평균 단어의 갯수 :',mean_decode_input_len)
print('target-output 한 문장안에서의 평균 단어의 갯수 :',mean_decode_target_len)

encode 한 문장안에서의 평균 단어의 갯수 : 45.0
target-input 한 문장안에서의 평균 단어의 갯수 : 42.0
target-output 한 문장안에서의 평균 단어의 갯수 : 42.0


In [60]:
# 10,000개의 문장에 대해
# 각 문장은 45개의 단어로 분리된다.

encoder_input_tr.size()

torch.Size([10000, 45])

In [61]:
decoder_input_tr.size()

torch.Size([10000, 42])

In [62]:
decoder_target_tr.size()

torch.Size([10000, 42])

## Train data

tuple type으로,  (encoder_input, decoder_input, decoder_target)의 순서를 가지게 됩니다. 이는 tuple assignment를 이용하려고 하는 것이니 자유롭게 사용하셔도 좋습니다.

In [64]:
train_data = tuple([encoder_input_tr, decoder_input_tr, decoder_target_tr])

## Eval data

지금까지 진행한 것은 학습 데이터에 대한 전처리입니다.

그런데 말입니다, 과연 평가 데이터는 학습데이터랑 동일한 형태일까요?

사용하는 단어도 동일할까요?

게다가 실제 데이터라면 문장 속에 이메일이나, 전화번호등 다른 것들로 대체해야할 게 있지 않을까요?

참으로 고민할 게 많습니다. 이 모든 것들을 고려하면 좋겠지만, 현재 과제에서는 학습 데이터의 단어 집합에는 없는데 평가 데이터에서 등장하는 단어들은 어떻게 처리할지 한번 고민해봅시다.

**핵심 TODO**:만약에 본 적이 없는 token이 나온다면 어떻게 될까요?

--> Out of Vocabulary !  =지금까지 단어 집합에 없던 아이들!<br>

이를 한번에 처리해주는 토큰으로 \<UNK\>(=unknown token) 을 사용

EX)

train data : "나는 밥을 먹는다."<br>
\>> ["나", "밥", "먹다"]<br>
\>> [\<SOS\>, 나, 밥, 먹다, \<EOS\>] <BR>
\>> {\<SOS\>:0, \<EOS\>:1, \<UNK\>: 2, 나:3, 밥:4, 먹다:5}  <bR>
\>> [0, 3, 4, 5, 1] <BR>

val data: "나는 고기를 먹는다"<br>
\>> ['나', '고기' '먹다']<BR>
\>> [\<SOS\>, 나, \<UNK\>, 먹다, \<EOS\>] <BR>
\>> [0, 3, 2, 5, 1] <BR><BR>

* 즉, 정리를 하자면, 우리는 학습데이터를 기준으로 학습을 하기에
* 평가를 할 시에 평가데이터에 있지만 학습데이터에는 없는 데이터에 한하여
* 어떤 처리를 할 것인가 <- 이게 새로운 문제이다.

* 그리고 여기에 대한 답으로 그냥 UNK(unknown)으로 새로운
* 정수로 라벨링을 해주는 것으로 대체할 것이다.

In [65]:
print(min(src_to_index.values()), max(src_to_index.values()))
print(min(tar_to_index.values()), max(tar_to_index.values()))

# source는 8865+1을 UNK으로
# target은 7293+1을 UNK로 설정한다.

1 8865
1 7293


In [66]:
# 조금 걸린다.

src_seq_val = [] # 전체 문장; 한국어
tar_seq_val = [] # 전체 문장; 영어

src_oov = [] # 기존의 train-data로부터 얻은 unique한 단어들 집합인 src_vocab에 없는 단어들
tar_oov = [] # 기존의 train-data로부터 얻은 unique한 단어들 집합인 tar_vocab에 없는 단어들


for raw_val_dat in tqdm(raw_val_data): # 평가 데이터인 'val'을 이용한다.
  
  # 1. 문장별 tokenization
  src_tmp = CustomTokenizer_ko(raw_val_dat['ko'])
  tar_tmp = CustomTokenizer_en(raw_val_dat['en'])


  # 2. 입력 시퀀스에서 문장의 끝을 알리는 <EOS> 토큰 추가; 한국어
  src_tmp = src_tmp + ['\n']

  # 출력 시퀀스에서 시작과 끝을 알리는 <SOS>, <EOS> 토큰 추가; 영어
  tar_tmp = ['\t'] + tar_tmp + ['\n']
  
 
  # 3. 처음보는 단어들은 일단 모르는 단어(OOV)로 표시해놓기  
  # 해당 과정에서는 지금 얻은 token들(src_tmp와 tar_tmp)들 중에서 학습 데이터의 단어 집합(src_vocab과 tar_vocab)에 없는 단어들을 찾아야 합니다.
  # 즉 위의 예시에서 "고기"에 해당하는 token을 찾아야 하고, 이를 list로 저장해봅시다.

  # 즉, raw_val_data에서 src_vocab과 tar_vocab
  src_oov += [x for x in src_tmp if x not in src_vocab and x not in src_oov]
  tar_oov += [x for x in tar_tmp if x not in tar_vocab and x not in tar_oov]


  # 4. OOV들은 <UNK>로 바꾸는 과정입니다.
  # 지금 문장을 tokenization되어 있고, 앞서 구한 모르는 단어(OOV)들의 리스트(src_oov와 tar_oov)가 있는 상황에서
  # 해당 리스트에 속한 토큰들은 <UNK>토큰으로 바꿔주는 것을 구현하는 파트입니다.
  # 위의 예시에서 본다면 [<SOS>, 나, 고기, 먹다, <EOS>]를 [<SOS>, 나, <UNK>, 먹다, <EOS>]로 만들어주는 과정입니다.
  src_tmp = list(map(lambda x: '<UNK>' if x in src_oov else x, src_tmp))
  tar_tmp = list(map(lambda x: '<UNK>' if x in tar_oov else x, tar_tmp))

 
  # 5. tokenization 끝난 문장 보관
  src_seq_val.append(src_tmp)
  tar_seq_val.append(tar_tmp)


100%|██████████| 1000/1000 [00:08<00:00, 123.17it/s]


In [67]:
# 나의 결과

tar_seq_val[5] # 아래의 정답과 일부 비슷하나 조금 다르다!

['\t', '<UNK>', '<UNK>', ',', '<UNK>', ',', 'and', '<UNK>', '<UNK>', '.', '\n']

정답 : 더블클릭해서 직접 확인!

['<SOS>', '<UNK>', '<UNK>', '<UNK>', 'and', 'even', '<UNK>', 'skin', '<EOS>']

많이 다른데.. 그 이유에 대해 확인해보자!

* ',' <- 이것은 내가 의도적으로 넣은 것이다.

* 따라서 even과 skin에 대해 살펴보자!

In [68]:
# 사용되는 영어문장

s = raw_val_data[5]['en']
s

'Provides smoother, brighter, and even-toned skin.'

In [69]:
# 영어 문장의 토큰화를 진행

tar_tmp = CustomTokenizer_en(s)
tar_tmp

['Provides',
 'smoother',
 ',',
 'brighter',
 ',',
 'and',
 'even-toned',
 'skin',
 '.']

In [70]:
# 이후에 train-data에 없는 단어이면, UNK로 대체

tar_tmp = list(map(lambda x: '<UNK>' if x in tar_oov else x, tar_tmp))
tar_tmp # <- 우리의 최종 결과

['<UNK>', '<UNK>', ',', '<UNK>', ',', 'and', '<UNK>', '<UNK>', '.']

In [71]:
'even' in tar_vocab # 'even'이 train-data에 있는가? <- yes!

True

In [72]:
'even' in tar_oov # 'even'이 test-data에 있는가? <- no!


False

즉, 'even-toned'으로 하이폰(-)으로 묶여있어 train에 이미 있어서 그대로 출력이 되어야하나 그러지 못한 것으로 확인이 된다.

In [73]:
'skin' in tar_vocab # 'skin'이 train-data에 있는가? <- no!

False

In [74]:
'skin' in tar_oov # 'skin'이 test-data에 있는가? <- yes!

True

즉, train data에서 어떤 이유로 skin이 나눠져서 단어로 들어가지 못했다. 그리고 이 때문에 차이가 발생했을 것으로 추정된다.

위의 샘플을 보면 생각보다 너무 많은 단어들이 UNK로 바뀌었습니다. 이게 과연 성능에 어떤 영향을 끼칠까요? 이를 방지할 수 있는 대책은 무엇일까요?

정답은 없습니다. 자유롭게 고민해보셔도 좋을 것 같습니다.

### UNK 토큰 추가 및 단어집합, token-index 업데이트

In [75]:
# UNK가 추가되면서 vocab과 index도 업데이트를 해야합니다.
# 즉 지금 방식으로 처리한다면, 지금 단어 집합에는 <UNK>라는 토큰이 없습니다.
# 그래서 UNK 토큰에도 정수를 할당해줘야 합니다.
# 지금 0은 padding에서 사용하고 있으니 아래의 과정은 마지막에 <UNK>를 추가해보는 것입니다.

In [76]:
# 입력 시퀀스의 마지막 토큰과 정수인코딩값 확인
list(src_to_index.items())[-1]

('힘쓰고', 8865)

In [77]:
# 현재 입력 시퀀스에 사용된 token들의 개수 확인
len(src_vocab)

8865

In [78]:
# 우리는 마지막 토큰 뒤에 UNK 토큰을 추가해봅시다
# 이 때 입력 시퀀스(src_to_index)와 출력시퀀스(tar_to_index)에 모두 사용해줘야 합니다.
src_to_index["<UNK>"] = len(src_vocab) + 1
tar_to_index["<UNK>"] = len(tar_vocab) + 1

In [79]:
# 입력시퀀스의 단어집합 확인
print(list(tar_to_index.items())[:5])
print(list(tar_to_index.items())[-1:-5:-1]) # 맨 마지막에 '<UNK>'가 있다.

[('\t', 1), ('\n', 2), ('!', 3), ('$', 4), ('%', 5)]
[('<UNK>', 7294), ('zoom', 7293), ('zones', 7292), ('zero-gravity', 7291)]


In [80]:
# 출력시퀀스의 단어집합 확인
print(list(src_to_index.items())[:5])
print(list(src_to_index.items())[-1:-5:-1]) # 맨 마지막에 '<UNK>'가 있다.

[('\n', 1), ('!', 2), ('"', 3), ('$10,000', 4), ('$10,000만', 5)]
[('<UNK>', 8866), ('힘쓰고', 8865), ('힘들어요', 8864), ('힘들어도', 8863)]


### 정수 인코딩

In [105]:
# 평가 데이터에 대해서  정수 인코딩을 진행해주세요! <- 즉, 테스트데이터를 정수형태로 바꾸라는 뜻!
# 이 과정에서 중요한 점은 우린 학습데이터만 가지고 있으니 학습 데이터에서 사용했던 token-index로 인코딩을 진행해야 합니다!


# 즉, 위에서 train-data에서 뽑은 unique한 단어들의 딕셔너리인 'src_to_index'를 이용한다.
# 여기에는 test-data의 단어는 들어가있지 않지만, test-data에서 '<UNK>'로 바꿔주었기에
# 이 값이 추가되었다.

# 정수 인코딩 값이 들어갈 장소
encoder_input_val = []
decoder_temp = []
decoder_input_val = []
decoder_target_val = []

# encoder의 입력값 정수 인코딩 진행
for sentence in src_seq_val:
  encoder_input_val.append(list(map(lambda x : src_to_index[x], sentence)))

# 우선 decoder에서 사용하는 문장에 대해 정수인코딩 진행
for sentence in tar_seq_val:
  decoder_temp.append(list(map(lambda x : tar_to_index[x], sentence)))

# 다만 decoder의 출력값은 <SOS>들어가면 안되니 첫번째 토큰은 빼주기
decoder_target_val = list(map(lambda x : x[1:], decoder_temp))
# decoder의 입력값은 마지막 <EOS>가 들어가면 안되니 마지막 토큰은 빼주기
decoder_input_val = list(map(lambda x : x[:-1], decoder_temp))

# 마지막으로 이제는 tensor의 형태로 바꿔준다.
encoder_input_val = list(map(lambda x : torch.Tensor(x).long(), encoder_input_val))
decoder_input_val = list(map(lambda x : torch.Tensor(x).long(), decoder_input_val))
decoder_target_val = list(map(lambda x : torch.Tensor(x).long(), decoder_target_val))

정수 레이블링이 잘 되었는지 확인해보자!

In [122]:
print('validation set에 관하여')
print('-'*50, '\n')

print('[Source]')
print('src_seq_val; 입력문장들을 토큰화한 이중 리스트의 길이 :', len(src_seq_val))
print('encoder_input_val; 토큰화한 문장들을 정수레이블한 이중 리스트의 길이 :', len(encoder_input_val))
print('\n')

print('[target]')
print('tar_seq_val; target문장들을 토큰화한 이중 리스트의 길이 :', len(tar_seq_val))
print('decoder_input_val; 토큰화한 문장들을 정수레이블한 이중 리스트의 길이 :', len(decoder_input_val))
print('decoder_target_val; 토큰화한 문장들을 정수레이블한 이중 리스트의 길이 :', len(decoder_target_val))

validation set에 관하여
-------------------------------------------------- 

[Source]
src_seq_val; 입력문장들을 토큰화한 이중 리스트의 길이 : 1000
encoder_input_val; 토큰화한 문장들을 정수레이블한 이중 리스트의 길이 : 1000


[target]
tar_seq_val; target문장들을 토큰화한 이중 리스트의 길이 : 1000
decoder_input_val; 토큰화한 문장들을 정수레이블한 이중 리스트의 길이 : 1000
decoder_target_val; 토큰화한 문장들을 정수레이블한 이중 리스트의 길이 : 1000


In [130]:
src_to_index['<UNK>'] # 입력 데이터에서 '<UNK>'의 정수레이블은 8866이다.

8866

In [123]:
tar_to_index['<UNK>'] # target 데이터에서 '<UNK>'의 정수레이블은 7294이다.

7294

In [124]:
tar_seq_val[0] # target 데이터의 1번째 문장을 토큰화하면 다음과 같다.

['\t',
 '<UNK>',
 'include',
 'heart',
 'rate',
 'monitoring',
 ',',
 '<UNK>',
 'battery',
 'life',
 ',',
 'music',
 'play',
 'control',
 'display',
 'sleep',
 'and',
 '<UNK>',
 'tracking',
 'waterproof',
 'smart',
 '<UNK>',
 '.',
 '\n']

In [125]:
src_seq_val[0] # source와 target의 순서가 맞게 되어있음을 확인했다.

['<UNK>',
 '모니터링',
 ',',
 '14일',
 '배터리',
 '<UNK>',
 ',',
 '음악',
 '플레이',
 '제어',
 '디스플레이',
 '수면',
 '및',
 '수영',
 '트래킹',
 '방수',
 '스마트',
 '알림',
 '기능',
 '이',
 '있습니다',
 '.',
 '\n']

In [131]:
decoder_input_val[0] # 각각 UNK -> 7294로 변경됨을 확인

tensor([   1, 7294, 3887, 3693, 5473, 4594,   16, 7294, 1548, 4243,   16, 4643,
        5132, 2310, 2714, 6135, 1277, 7294, 6767, 7126, 6153, 7294,   17])

In [128]:
encoder_input_val[0] # 각각 UNK -> 8866로 변경됨을 확인

tensor([8866, 3041,   18,   95, 3468, 8866,   18, 6005, 8211, 6871, 2478, 4479,
        3247, 4495, 7945, 3421, 4574, 5127, 1352, 6041, 6390,   20,    1])

### Padding

In [132]:
# 현재 길이가 제각각인 정수화된 문장이 있습니다. padding을 통해 길이를 맞춰봅시다
# 주의사항: 여기에 사용된 값을은 정수일까요?
batch_first = True

# _val이 붙으면, validation 즉, test-data이다.
encoder_input_val = pad_sequence(encoder_input_val, batch_first=batch_first).long()
decoder_input_val = pad_sequence(decoder_input_val, batch_first=batch_first).long()
decoder_target_val = pad_sequence(decoder_target_val, batch_first=batch_first).long()

In [133]:
decoder_target_val[0] # 0이 채워진 것 확인

tensor([7294, 3887, 3693, 5473, 4594,   16, 7294, 1548, 4243,   16, 4643, 5132,
        2310, 2714, 6135, 1277, 7294, 6767, 7126, 6153, 7294,   17,    2,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

In [134]:
val_data = tuple([encoder_input_val, decoder_input_val, decoder_target_val])

# 3. Dataset

실습코드에서 사용한 것과 동일한 구조입니다! 

사실 앞서 처리가 진행되었던 모든 과정은 Dataset에서 __get_item__ 함수 내에서 처리해서 값을 리턴할 수도 있습니다.

다만 설명을 위해 이번 과제에서는 과정을 단계별로 설명 드렸습니다. 추후에 진행하실 때엔 __get_item__에서 구현해보셔도 좋을 것 같아요!

그리고 일부 단계의 경우 임의로 처리된 것들이 있습니다.(tokenization부터 stopwords 등등) 가령 이메일, 전화번호와 같은 단어들은 \<UNK\>처럼 새로운 토큰을 이용해서 처리할 수도 있으며 목적에 따라서 다른 tokenziation을 사용할 수 있습니다.

**(참고)**

 __get_item__말고 사전에 전처리를 다한 상태로 사용할 경우 사실 사용할 모든 데이터를 메모리에 올려놓고 사용하는 것과 같습니다.(변수에 할당되어 있으니)

그런데 우리가 데이터가 매매매매매우 많아진다면, 메모리가 부족해질 수도 있지 않을까요? 그렇기 때문에 때에 따라 data를 필요할 때 전처리를 하고 반환하기도 합니다. 여기서 data가 필요할때란 결국 __get_item__을 호출할 때가 될 것입니다.

정리하자면, 꼭 모든 데이터를 한번에 다 전처리해놓도 준비해놓지 않고도, 필요할 때 마다(get_item이 호출될 때마다) 전처리해서 결과를 전달하는 방법도 있습니다.

다만 해당 과제에서는 Seq2Seq에 익숙해지는 것을 목표로 하기에 이러한 점만 알고 가셔도 충분하기도 하고, 오히려 사전에 다 처리해놓는 현재 방식이 더 효율적일 수도 있습니다. 그렇기 때문에 이 점만 알고 가셔도 좋을 것 같습니다!

In [135]:
from torch.utils.data import Dataset, DataLoader

In [136]:
class textDataset(Dataset):
  def __init__(self, data, batch_first=True):

    # 상속
    super(textDataset, self).__init__()

    # tuple asgginment를 활용한다면 굳이 argument로 모든 데이터를 받지 않아도 됩니다!
    enc_inp, dec_inp, dec_out = data

    # 내부에서 사용할 변수들 정리하는 것으로, 모든 데이터가 torch.LongTensor로 정리된 값들이 저장됩니다.
    self.enc_inp = enc_inp
    self.batch_first= batch_first # 이전에 True로 설정
    self.dec_inp = dec_inp
    self.dec_out = dec_out

  # 함수 
  def __getitem__(self, idx):
     return self.enc_inp[idx], self.dec_inp[idx],self.dec_out[idx]

  def __len__(self):
    return self.enc_inp.size(int(~self.batch_first))

In [137]:
train_dataset = textDataset(train_data, batch_first=batch_first)
val_dataset = textDataset(val_data, batch_first=batch_first)

In [138]:
train_dataset[0]

(tensor([1219, 2837, 5996, 2454, 1390,   18, 6634, 5691,  774, 8306,  861, 5996,
          979, 8497, 7053, 8476,   20,    1,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0]),
 tensor([   1,  573,   10, 3540, 6712, 3689, 6636,   16, 1277,  573, 3766, 7281,
         2744, 2253, 2757, 1789, 7213, 6964,   17,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]),
 tensor([ 573,   10, 3540, 6712, 3689, 6636,   16, 1277,  573, 3766, 7281, 2744,
         2253, 2757, 1789, 7213, 6964,   17,    2,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]))

In [139]:
val_dataset[0]

(tensor([8866, 3041,   18,   95, 3468, 8866,   18, 6005, 8211, 6871, 2478, 4479,
         3247, 4495, 7945, 3421, 4574, 5127, 1352, 6041, 6390,   20,    1,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]),
 tensor([   1, 7294, 3887, 3693, 5473, 4594,   16, 7294, 1548, 4243,   16, 4643,
         5132, 2310, 2714, 6135, 1277, 7294, 6767, 7126, 6153, 7294,   17,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]),
 tensor([7294, 3887, 3693, 5473, 4594,   16, 7294, 1548, 4243,   16, 4643, 5132,
         2310, 2714, 6135, 1277, 7294, 6767, 7126, 6153, 7294,   17,    2,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]))

# 4. Model

In [140]:
import torch.nn as nn
import torch.functional as F

## Encoder

본 모델에서는 LSTM를 기본 단위로 가지는 Seq2Seq 모델을 구성할 것

참고 [레퍼런스](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)

In [156]:
class Encoder(nn.Module):


  # 변수선언
  def __init__(self, in_size, hid_size, tok_size, bat_first=True, num_lay=1, bidirect=False):
    super(Encoder, self).__init__()
    # 여기서 hidden_size는 LSTM의 hidden_size로 embedding vector를 몇차원의 벡터로 transform할 것인지 말합니다!
    self.hidden_size = hid_size
   
    # 앞서 설명한 내용대로 전체 사용하는 token의 수를 embedding_num에 해당하고, 이를 몇차원의 embedding dim으로 표현할 것인지 말합니다.
    # 이 때 LSTM의 관점에서 embedding dim은 input_size에 해당해서 개념적으로 embedding_dim=in_size입니다.
    self.embedding = nn.Embedding(tok_size, in_size)

    # layer를 구성할 때 LSTM 내부의 layer의 개수도 조정할 수 있습니다.
    self.num_layers = num_lay

    # 이것은 bidirectional 방식을 사용할 것인지를 얘기하는데,
    # 이는 현재 과정에서 고려하지 않는 값이지만 LSTM 레이어를 설정할 때 필요한 값이라 기록하긴 했습니다.
    self.num_directional = 2 if bidirect else 1

    self.batch_first = bat_first
    self.lstm = nn.LSTM(in_size, 
                        hid_size, 
                        num_layers = num_lay,
                        batch_first = bat_first,
                        bidirectional=bidirect)



  # 함수선언
  def forward(self, x, hidden):
    """
    현재 전체 전처리부터 모델링 과정을 batch_first=True환경에서 진행하고 있으니 이를 유의하시길 바랍니다.
    batch_first=True이기에 입력값(x)의 차원은 다음과 같고, 
    hidden_state에 해당하는 값도 batch_size가 먼저 나오게 됩니다.
    x: (batch_size, seq_len)
    hidden: (num_directional*num_layers, batch_size, hidden_size)
    
    개념적으로 보면 우리가 전처리한 시퀀스(seq)는
    seq = [정수인코딩1, 정수인코딩2, ..]에 해당합니다.
    여기서 중요한 것은 들어오는 값이 정수인코딩된 값입니다!!
    
    이런 seq들이 모여서 만들어진 것이 
    x = [seq1, seq2, ...]에 해당합니다.

    그래서 len(x) = batch_size가 되는 것입니다!    
    """

    # 여기서 중요한 점은 입력값으로 정수인코딩된 값이 들어오는 것이고, embedidng layer를 거치면서 
    # 정수인코딩값이 vector로 바뀌게 됩니다!!
    # 그런데 이상하지 않나요? 위에서 one-hot된 벡터를 입력해서 embedding vector로 바꿔주는데
    # 정수인코딩 값만 넣다니?!
    # 이는 사실 torch의 Embedding layer가 그런 역할까지 포함하고 있는 것이고, 
    # Embedding layer를 사용할 땐 "정수"인코딩된 값을 넣어주면 대응하는 embedding vector를 반환해줍니다.
    # 여기서 "정수"인코딩 값을 사용해야 하기 때문에 앞에서 .long()과정이 들어간 것입니다.
    # 안그럼 에러뜰거에요
    # emb : (batch_size, seq_len, input_size)
    emb = self.embedding(x)

    # LSTM에서 하는 연산은 결국 Linear와 크게 다르지 않습니다.
    # input_size의 임베딩 벡터를 hidden_size의 벡터로 바꾸게 되니 output의 차원은
    # out : (batch_size, seq_len, hidden_size)

    # 이와 달리 hidden_state는 그 구조가 조금 다릅니다.
    # 여기서 num_layer가 사용되는 것은 LSTM에서 내부의 layer가 많아질수록 그 만큼 hidden state가 더 나와야 할 것입니다.
    # 위에 LSTM layer에서 인용한 사진이 잘 설명하는데 내부에서 layer가 많아질수록 hidden state도 많아지게 될 것입니다.
    # 그리고 최종 반환하는 hidden_state는 앞의 시퀀스의 정보를 다 포함하는 벡터가 나온 것이고,
    # 그 모든 정보를 가지고 있는 벡터의 차원이 바로 hidden_size입니다.
    # 즉 layer마다 batch 만큼의 context vector가 담겨져 있다! 생각할 수 있을거에요
    # hidden: (num_layers, batch_size, hidden_size)
    out, hidden = self.lstm(emb, hidden)
    return out, hidden

## Decoder

decoder의 hidden이 아닌 out으로 예측하고, 그 값 하나를 이용해서 다음 값 계산에 사용하기 때문에 seq_len =1이 된다.

In [157]:
class Decoder(nn.Module):
  def __init__(self, in_size, hid_size, tok_size, bat_first=True, num_lay=1, bidirect=False):
    super(Decoder, self).__init__()

    # 대부분의 방식은 사실 Encoder와 비슷하게 흘러갑니다.


    # 변수선언

    # LSTM이 embedding을 몇차원의 벡터로 변환할지 정하는 값입니다.
    self.hidden_size = hid_size

    # 여기서도 번역해야하는 언어의 token을 embedding 해야하고 이  때 embedding vector의 차원이 결국 LSTM입장에서의 Input size가 됩니다.
    self.embedding = nn.Embedding(tok_size, in_size)

    self.num_layers = num_lay
    self.num_directional = 2 if bidirect else 1
    self.batch_first = bat_first
    self.lstm = nn.LSTM(in_size, 
                        hid_size, 
                        num_layers = num_lay,
                        batch_first = bat_first,
                        bidirectional=bidirect)
    self.relu =  nn.ReLU()

    # 여기서 Encoder와 차이가 생기는데,
    # Decoder는 입력된 단어와 context를 보고 다음 단어를 "예측"해야합니다.
    # 그렇기 때문에 hidden_size의 벡터를 token의 개수만큼 다시 mapping해주어야
    # 각 token별 점수로 만들어줘야 최종 예측을 할 수 있습니다.
    # 그래서 최종 값을 반환할 때에는 총 토큰 수만큼으로 만들어 줘야 토큰별 확률을 계산하게 됩니다.
    self.fc1 = nn.Linear(hidden_size, tok_size)
    



  # 함수선언
  def forward(self, x, hidden):
    """
    x: (batch_size, seq_len=1)
    hidden: (num_directional*num_layers, batch_size, hidden_size)
    """
    
    # 여기서 조금 trivial한 점을 고려해야 합니다.
    # Embedding layer를 거치고 나왔을 때 차원은 아래와 같아야 하고 이를 위해서 x[:, 0]과 .unsqueeze(1)을 한 것입니다. 이것은 정해진 답은 없고, 필요하신 대로 이용하셔도 좋습니다!
    # emb: (batch_size, seq_len=1, input_size)   
    # x[:, 0] : (batch_size, )
    # self.embedding(x[:, 0]) : (batch_size, input_size)
    # self.embedding(x[:, 0]).unsqueeze(1) : (batch_size, 1, input_size)
    emb = self.embedding(x[:, 0]).unsqueeze(1) # <- 이렇게 해서 차원을 1차원으로 만들어준다.
    # 이거 말고도 동일한 차원을 가지도록 처리하려면 어떻게 해야할까요?
    # emb = self.embedding(~~)~~

    # relu를 취한 것은 별다른 이유가 없이 참고한 레퍼런스와 동일한 구조로 해본 것입니다.
    out = self.relu(emb)

    # LSTM을 사용한다는 점에서 Encoder와 동일하게 output과 hidden_state는 동일한 구조로 반환하게 됩니다.
    # 다만 seq_len은 decoder에서 1이 된다는 점입니다!
    # out : (batch_size, seq_len=1, hidden_size)
    # hidden: (num_layers, batch_size, hidden_size)
    out, hidden = self.lstm(out, hidden)

    # 기존의 output에서 seq_len =1이니 squeeze하게 된다면 (batch_size, hidden_size)가 되고
    ## out.squeeze(1) : (batch_size, hidden_size)
    # 이 상태에서 FC layer를 통해 최종 토큰 개수 만큼 값을 반환하게 됩니다.
    # out: (batch_size, tok_size) : batch의 element별로 token에 대해 예측값을 반환
    out = self.fc1(out.squeeze(1))
    return out, hidden

In [158]:
def init_hidden(self, x):
  """
  레퍼런스 참고할 때 hidden_state 구조와 동일하게 되어야 합니다.
  여기서 보면 
  when batch_first=True (num_directional*num_layers, batch_size, hidden_size)
  """
  batch_size = x.size(0) if self.batch_first else x.size(1)
  h0 = torch.zeros(self.num_layers*self.num_directional, batch_size, self.hidden_size).to(device)
  c0 = torch.zeros(self.num_layers*self.num_directional, batch_size, self.hidden_size).to(device)
  return h0, c0

## Seq2Seq

In [159]:
class Seq2Seq(nn.Module):
  def __init__(self, enc, dec):
    super(Seq2Seq, self).__init__()
    assert enc.hidden_size == dec.hidden_size
    assert enc.num_layers == dec.num_layers

    self.encoder = enc
    self.decoder = dec




  def forward(self, enc_inp, dec_inp, use_teacher_force=True):
    """
    enc_inp : (batch_size, enc_seq_len)
    dec_inp : (batch_size, dec_seq_len)
    """

    batch_size = enc_inp.size(0)
    enc_seq_len = enc_inp.size(1)
    dec_seq_len = dec_inp.size(1)
    # 최종 반환하는 것은 "토큰 별" score 값이니 이 점을 이용해서 token 개수 확인
    token_size =  self.decoder.fc1.out_features

    # decoder의 결과값은 출력 시퀀스에서 각 위치에 들어갈 token들의 확률(점수) 예측값 저장할 것입니다.
    # 그래서 구조는 (batch_size, seq_len, token_size)가 됩니다.
    # 즉 decoder는 step 마다 한 토큰씩 예측하게 되고 이를 총 seq_len만큼 진행해서 모으게 됩니다.
    # 이를 저장할 변수는 outputs
    # 이 때 seq_len은 decoder의 seq_len에 해당
    outputs = torch.zeros(batch_size, dec_seq_len, token_size)

    # Step 0: Encoder의 forward에 필요한 초기 hidden, cell state 계산
    enc_init_hidden = init_hidden(self.encoder, enc_inp)

    # Step 1: Encoder를 이용하여 context 벡터 계산
    # 중요한 점은 Encoder의 output이 아닌, hidden_state에 해당한다!!!
    _, context = self.encoder(enc_inp, enc_init_hidden)

    # Step 2: Decoder의 초기 입력값을 먼저 확인
    # 여기서 Decoder의 입력값이란 context vector 말고 단어들을 말합니다.
    # 그리고 첫 입력값은 <SOS>가 사용되어야 한다는 것
    # 이는 decoder의 input에서 첫번째 값에 해당하고 지금 상황에서는 앞서 사용한 \t을 알려주는 토큰이 사용되어야 합니다.
    # 그리고 Decoder의 인풋은 위에서 구현할 때  x: (batch_size, seq_len=1) 라고 했고 이 때 seq_len은 1이 됩니다.
    # 즉 이 구조를 맞춰서 넣어줘야 돌아가기에
    # (batch_size, dec_seq_len)의 차원을 맞춰야 하고, dec_seq_len=1이라서 마지막 차원에 unsqueeze 진행
    dec_inp_t = dec_inp[:, 0].unsqueeze(-1)

    # Step 3: Decoder를 이용하여 하나하나 계산에 사용
    # Encoder의 최종 리턴한 hidden state가 context 벡터고 이게 decoder입장에서보면 init_hidden에 해당합니다.
    # 또한 앞서 첫 입력값은 직접 할당했으니 전체 loop 시작은 1부터 하면 됩니다.
    # loop의 순서는 output 계산 --> 예측 --> 다음 입력값 업데이트
    # 이 순서라서 range는 1부터 시작하게 만들면 됩니다.
    dec_hidden = context
    for t in range(1, dec_seq_len):
      out, dec_hidden = self.decoder(dec_inp_t, dec_hidden)
      # 각 step별 결과물 = 해당 시점에서 각 토큰별 등장 확률(점수)값 기록
      outputs[:, t, :] = out

      if use_teacher_force:
        # teacher force를 사용하는 경우에는 실제 input에서 입력값을 넣어야 해서 decoder의 input값을 이용해서 업데이트를 합니다.
        # 그리고 모델이 돌아가기 위해서 차원도 차원 맞출 필요 존재힙니다.
        # 즉, indexing하게 되버리면 dec_inp차원과 맞지 않아서 unsqueeze 진행
        dec_inp_t = dec_inp[:, t].unsqueeze(-1)
      else:
        # keepdim을 통해 차원수 유지해줘서 구조를 동일하게 맞춘다.
        dec_inp_t = out.argmax(1, keepdim=True)

    return outputs

# 5. Setup

In [145]:
# padding에 해당하는 token도 고려하기
enc_token_size = len(src_to_index)+1
dec_token_size = len(tar_to_index)+1

# Hyper-parameter!!
# RAM 메모리 이슈로 다운 되는 경우 때문에 줄였습니다.
input_size = 4
hidden_size = 4

In [146]:
encoder = Encoder(input_size, hidden_size, enc_token_size)
decoder = Decoder(input_size, hidden_size, dec_token_size)

model = Seq2Seq(encoder, decoder).to(device)

## 생략된 내용들

모델 생성하는 과정에서 초기값이나 등등 부수적인 부분들은 현재 과제 코드상에서 생략되었습니다. 모델 개선에 관심있으신 분들은 수행하셔도 좋습니다.

In [147]:
# ~~~

# 6. Train

In [160]:
def train(model, loader, optimizer, cri):
  model.train()
  loss_ep = 0

  for enc_input, dec_input, dec_target in loader:
    optimizer.zero_grad()

    # 여기서 output은 seq2seq 모델의 결과값임을 유의해야합니다!
    # output: (batch_size, seq_len, token_size)
    output = model(enc_input.to(device), dec_input.to(device), use_teacher_force=True)
    token_size = output.size(-1)

    # 지금 [문장, 문장, ...] 구조에서 문장들을 그냥 순서대로 이어 붙인 것
    # 이는 loss 계산을 쉽게 하기 위해 구조를 조정
    # output : (batch_size * seq_len, token_size)
    output = output.view(-1, token_size)

    # 위와 동일한 방식으로 label 값도 동일한 구조로 만들어주려고 view 실행
    # dec_target: (batch_size, dec_seq_len)
    # 즉 그냥 1열로 다 펴 버리면 CrossEntropyloss를 위한 구조 완성
    # target L (batch_size*dec_seq_len,)
    target = dec_target.view(-1)

    loss = cri(output, target)

    loss.backward()
    optimizer.step()

    loss_ep += loss.item()
  return loss_ep/len(loader)

# 7. Evaluation

In [149]:
import pandas as pd

In [161]:
def val(model, loader, cri):
  # 평가할 땐 loss말고 예측한 결과도 저장할 것
  # 이것이 바로 result
  result = torch.Tensor() # <- 여기에 결과를 저장

  model.eval()
  loss_ep = 0
  with torch.no_grad():
    for enc_input, dec_input, dec_target in loader:
      # output: (batch_size, seq_len, token_size)
      output = model(enc_input.to(device), dec_input.to(device), use_teacher_force=False)

      # 예측 값을 저장
      decode_idx = output.argmax(2).detach().cpu()
      # 각 시점별 예측 결과를 저장
      result = torch.cat((result, decode_idx), dim=0)
  
      # 지금 [문장, 문장, ...] 구조에서 문장들을 그냥 순서대로 이어 붙인 것
      # loss 계산을 위한 조정
      # train과 동일한 이유 
      token_size = output.size(-1)
      output = output.view(-1, token_size)

      # 위와 동일한 방식으로 label 값도 조정
      # dec_target: (batch_size, dec_seq_len)
      # 즉 그냥 1열로 다 펴 버리면 CrossEntropyloss를 위한 구조 완성
      # target L (batch_size*dec_seq_len,)
      target = dec_target.view(-1)

      loss = cri(output, target)
      loss_ep += loss.item()
  return loss_ep/len(loader), result

In [151]:
# 아래의 dictionary는 정수 인코딩 값을 넣으면 해당하는 원래 알파벳이 나옴
# padding도 처리해야하기 때문에 0은 그냥 ''으로 처리
# 이는 예측한 정수 인코딩 값을 실제 단어로 바꾸기 위한 것이라서 key, value를 바꿔주는 방식
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())
index_to_tar[0] = ''
index_to_src[0] = ''

In [162]:
def decode(model, sample):
  # 정수 인코딩된 것을 실제 단어로 바꾸는 과정이고,과제 수행에 중요한 것은 아님
  enc_input, dec_input, dec_output = sample
  enc_input = enc_input.unsqueeze(0)
  dec_input = dec_input.unsqueeze(0)
  
  model.eval()
  with torch.no_grad():
    output = model(enc_input.to(device), dec_input.to(device), use_teacher_force=False)
    decode_idx = output.argmax(2)

  sentence_inp = pd.Series(enc_input.squeeze(0).detach().cpu().numpy()).apply(lambda x: index_to_src[x])
  sentence_out = pd.Series(decode_idx.squeeze(0).detach().cpu().numpy()).apply(lambda x: index_to_tar[x])
  sentence_act = pd.Series(dec_output.detach().cpu().numpy()).apply(lambda x: index_to_tar[x])

  print(f"입력 문장({len(sentence_inp)}) : {''.join(sentence_inp.tolist())}")
  print(f"실제 문장({len(sentence_act)}) : {''.join(sentence_act.tolist())}")
  print(f"예측 문장({len(sentence_out)}) : {''.join(sentence_out.tolist())}")

In [153]:
import time
from tqdm import tqdm
from torch import optim

# 8. Run

In [154]:
train_dataset[0]

(tensor([1219, 2837, 5996, 2454, 1390,   18, 6634, 5691,  774, 8306,  861, 5996,
          979, 8497, 7053, 8476,   20,    1,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0]),
 tensor([   1,  573,   10, 3540, 6712, 3689, 6636,   16, 1277,  573, 3766, 7281,
         2744, 2253, 2757, 1789, 7213, 6964,   17,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]),
 tensor([ 573,   10, 3540, 6712, 3689, 6636,   16, 1277,  573, 3766, 7281, 2744,
         2253, 2757, 1789, 7213, 6964,   17,    2,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]))

In [163]:
# 학습은 아래의 코드를 이용하여 진행

train_lodaer = DataLoader(train_dataset, batch_size=1024)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)

optimizer = optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()

train_loss = []
val_loss = []

epochs = 10
for epoch in tqdm(range(epochs)):
  start = time.time()

  train_loss_ep = train(model,train_lodaer,optimizer, criterion)
  val_loss_ep, val_token = val(model, val_loader, criterion)

  train_loss.append(train_loss_ep)
  val_loss.append(val_loss_ep)
  if epoch % 2 == 1:
    print(f"Epoch : {epoch}")
    print(f"Train Loss : {train_loss_ep:.4f}")
    print(f"Val Loss : {val_loss_ep:.4f}")
    decode(model, val_dataset[100])


 20%|██        | 2/10 [05:21<21:25, 160.72s/it]

Epoch : 1
Train Loss : 7.2871
Val Loss : 7.0245
입력 문장(41) : 저는<UNK><UNK>스마트<UNK><UNK><UNK>및온도센서<UNK>사용무료앱제품을소개하려고합니다.

실제 문장(36) : Iwouldliketointroducea<UNK><UNK>smart<UNK><UNK><UNK>andtemperaturesensoreasytousefreeappproduct.

예측 문장(36) : youryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryouryour


 40%|████      | 4/10 [10:42<16:02, 160.48s/it]

Epoch : 3
Train Loss : 5.5444
Val Loss : 5.4227
입력 문장(41) : 저는<UNK><UNK>스마트<UNK><UNK><UNK>및온도센서<UNK>사용무료앱제품을소개하려고합니다.

실제 문장(36) : Iwouldliketointroducea<UNK><UNK>smart<UNK><UNK><UNK>andtemperaturesensoreasytousefreeappproduct.

예측 문장(36) : ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


 60%|██████    | 6/10 [16:02<10:41, 160.42s/it]

Epoch : 5
Train Loss : 4.0637
Val Loss : 4.1753
입력 문장(41) : 저는<UNK><UNK>스마트<UNK><UNK><UNK>및온도센서<UNK>사용무료앱제품을소개하려고합니다.

실제 문장(36) : Iwouldliketointroducea<UNK><UNK>smart<UNK><UNK><UNK>andtemperaturesensoreasytousefreeappproduct.

예측 문장(36) : 


 80%|████████  | 8/10 [21:29<05:23, 161.70s/it]

Epoch : 7
Train Loss : 2.9962
Val Loss : 3.4075
입력 문장(41) : 저는<UNK><UNK>스마트<UNK><UNK><UNK>및온도센서<UNK>사용무료앱제품을소개하려고합니다.

실제 문장(36) : Iwouldliketointroducea<UNK><UNK>smart<UNK><UNK><UNK>andtemperaturesensoreasytousefreeappproduct.

예측 문장(36) : 


100%|██████████| 10/10 [26:49<00:00, 160.99s/it]

Epoch : 9
Train Loss : 2.6424
Val Loss : 3.2739
입력 문장(41) : 저는<UNK><UNK>스마트<UNK><UNK><UNK>및온도센서<UNK>사용무료앱제품을소개하려고합니다.

실제 문장(36) : Iwouldliketointroducea<UNK><UNK>smart<UNK><UNK><UNK>andtemperaturesensoreasytousefreeappproduct.

예측 문장(36) : 


보완할 점
* 왠지 모르게 띄어쓰기가 없다.
* 예측 문장에서 같은 단어가 계속 반복된다.
* 토큰화를 할시에 , 같은 것들을 제거해주고
* '놀았다 -> 놀다'와 같은 옵션을 넣어주어야할 것 같다.

In [164]:
# 학습을 진행한 다음, 모델 백업해놓기
torch.save(model.state_dict(), 'model_hw.pt')